In [2]:
path = r'C:\Users\MGroup\Downloads\Re Offer №453757 13.11.2023, Contrinex _ FAMAGA Group GmbH & Co. KG .eml'
dir =  r'notebooks/eml_parser.ipynb'

In [18]:
os.getcwd()

'C:\\Users\\MGroup\\components_agent_sales\\notebooks'

In [4]:
# with open(path, 'r') as f:
#     print(f.read())

In [6]:
import os
from email import message_from_file

def extract_email_data(eml_file):
    # Use the existing script or modified version to extract data from eml_file
    # Return a dictionary containing the email's content and metadata
    # Example: {"timestamp": ..., "subject": ..., "sender": ..., "recipient": ..., "content": ...}
    pass

def process_directory(directory_path):
    email_data = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.eml'):
            with open(os.path.join(directory_path, file_name), 'rb') as f:
                email_info = extract_email_data(f)
                email_data.append(email_info)
    
    # Sort the emails based on your criteria, e.g., timestamp
    sorted_emails = sorted(email_data, key=lambda x: x['timestamp'])

    return sorted_emails

# Directory containing your .eml files
# Process the directory and get sorted emails
sorted_emails = process_directory(path)

# Output or process the sorted emails as needed
for email in sorted_emails:
    print(email)


NotADirectoryError: [WinError 267] Неверно задано имя папки: 'C:\\Users\\MGroup\\Downloads\\Re Offer №453757 13.11.2023, Contrinex _ FAMAGA Group GmbH & Co. KG .eml'

In [20]:
from email import policy
from email.parser import BytesParser
from email.message import EmailMessage
import os

def extract_email_data(eml_file_path) -> dict:
    """
    Extracts data from an .eml file.

    Args:
    eml_file_path (str): Path to the .eml file.

    Returns:
    dict: Extracted email data including sender, recipient, subject, date, and body.
    """

    with open(eml_file_path, 'rb') as f:
        msg: EmailMessage = BytesParser(policy=policy.default).parse(f)

    # Extracting basic details
    email_data = {
        "from": msg["From"],
        "to": msg["To"],
        "subject": msg["Subject"],
        "date": msg["Date"]
    }

    # Extracting the body of the email
    if msg.is_multipart():
        parts = [part.get_payload(decode=True) for part in msg.walk() if part.get_content_type() == 'text/plain']
        body = ''.join(str(part, 'utf-8') for part in parts)
    else:
        body = str(msg.get_payload(decode=True), 'utf-8')

    email_data["body"] = body.strip()

    return email_data

In [ ]:
from email.parser import BytesParser
from email.message import EmailMessage
from email import policy

with open(path, 'rb') as f:
    msg: EmailMessage = BytesParser(policy=policy.default).parse(f)
    parts = [part.get_payload(decode=True) for part in msg.walk() if part.get_content_type() == 'text/plain']
    body = ''.join(str(part, 'utf-8') for part in parts)
    print(body)

In [ ]:
print(extract_email_data(path)['body'])

In [11]:
def separate_messages(email_body):
    """
    Separates individual messages from an email thread.

    Args:
    email_body (str): The body of the email containing the entire conversation.

    Returns:
    list: A list of individual email messages.
    """
    # Split the email body into lines
    lines = email_body.split('\n')

    # Initialize variables
    messages = []
    current_message = []

    # Iterate over each line
    for line in lines:
        if line.startswith('>'):
            # Start of a quoted message
            if current_message:
                # Add the current message to the list and start a new one
                messages.append('\n'.join(current_message).strip())
                current_message = []
        else:
            # Part of the current message
            current_message.append(line)

    # Add the last message if it exists
    if current_message:
        messages.append('\n'.join(current_message).strip())

    return messages

# Example Usage
email_body = "..."  # Replace with your actual email body
messages = separate_messages(email_body)
for message in messages:
    print("Message:")
    print(message)
    print("------------")

Message:
...
------------


In [12]:
separate_messages(extract_email_data(path)['body'])

['Good day,\n\nVery dear Mr. Lesar, thanks you for your order! Proforma invoice for \npayment attached. Please send me a payment confirmation after it will be \nmade to start processing your order as fast as possible.\n\nWish you a nice day!\n\n-- \nMit freundlichen Grüßen / Best regards / Z wyrazami szacunku,\nSawwa Wronskiy\nVertrieb / Sales Manager\nFAMAGA Group OHG\n\n+48226022408\nhttps://famaga.de/',
 '-- \nMit freundlichen Grüßen / Best regards / Z wyrazami szacunku,\nSawwa Wronskiy\nVertrieb / Sales Manager\nFAMAGA Group OHG\n\n+48226022408\nhttps://famaga.de/']

In [13]:
from email import policy
from email.parser import BytesParser
with open(path, 'rb') as fp:
    msg = BytesParser(policy=policy.default).parse(fp)

print('To:', msg['to'])
print('From:', msg['from'])
print('Subject:', msg['subject'])

To: "vzdrževanje | Eurobox d.o.o." <vzdrzevanje@eurobox.si>
From: Sawwa Vronskiy <ws@famaga.de>
Subject: Re: Offer №453757 13.11.2023, Contrinex || FAMAGA Group GmbH & Co. KG *


In [15]:
import sys
import os
import os.path
from collections import defaultdict
from email.parser import Parser

eml_mail = 'your eml file'
output_dir = 'mention the directory where you want the files to be download'

def parse_message(filename):
    with open(filename) as f:
        return Parser().parse(f)

def find_attachments(message):
    """
    Return a tuple of parsed content-disposition dict, message object
    for each attachment found.
    """
    found = []
    for part in message.walk():
        if 'content-disposition' not in part:
            continue
        cdisp = part['content-disposition'].split(';')
        cdisp = [x.strip() for x in cdisp]
        if cdisp[0].lower() != 'attachment':
            continue
        parsed = {}
        for kv in cdisp[1:]:
            key, val = kv.split('=')
            if val.startswith('"'):
                val = val.strip('"')
            elif val.startswith("'"):
                val = val.strip("'")
            parsed[key] = val
        found.append((parsed, part))
    return found

def run(eml_filename, output_dir):
    msg = parse_message(eml_filename)
    attachments = find_attachments(msg)
    print ("Found {0} attachments...".format(len(attachments)))
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    for cdisp, part in attachments:
        cdisp_filename = os.path.normpath(cdisp['filename'])
        # prevent malicious crap
        if os.path.isabs(cdisp_filename):
            cdisp_filename = os.path.basename(cdisp_filename)
        towrite = os.path.join(output_dir, cdisp_filename)
        print( "Writing " + towrite)
        with open(towrite, 'wb') as fp:
            data = part.get_payload(decode=True)
            fp.write(data)

In [19]:
run(path, os.getcwd())

Found 1 attachments...
Writing C:\Users\MGroup\components_agent_sales\notebooks\Proforma_453757.pdf


In [ ]:
import email

with open(path) as email_file:
    email_message = email.message_from_file(email_file)

print(email_message.get_payload()[0])